In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<p style = "font-size : 50px; color : #055ff0 ; font-family : 'Chalkduster',fantasy; text-align : center; background-color : #d6facf; border-radius: 5px 5px;"><strong>House Price Prediction</strong></p>


<a id = '1.0'></a>
<p style = "font-size : 30px; color :#090f01 ;font-family : 'Chalkduster',fantasy;  "><strong>Importing required libraries :-</strong></p>

In [ ]:
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
train = pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")
test = pd.read_csv("../input/house-prices-advanced-regression-techniques/test.csv")
import warnings
warnings.filterwarnings('ignore')

The features in the given dataset are:.

* SalePrice - the property's sale price in dollars. This is the target variable that you're trying to predict.
* MSSubClass: The building class
* MSZoning: The general zoning classification
* LotFrontage: Linear feet of street connected to property
* LotArea: Lot size in square feet
* Street: Type of road access
* Alley: Type of alley access
* LotShape: General shape of property
* LandContour: Flatness of the property
* Utilities: Type of utilities available
* LotConfig: Lot configuration
* LandSlope: Slope of property
* Neighborhood: Physical locations within Ames city limits
* Condition1: Proximity to main road or railroad
* Condition2: Proximity to main road or railroad (if a second is present)
* BldgType: Type of dwelling
* HouseStyle: Style of dwelling
* OverallQual: Overall material and finish quality
* OverallCond: Overall condition rating
* YearBuilt: Original construction date
* YearRemodAdd: Remodel date
* RoofStyle: Type of roof
* RoofMatl: Roof material
* Exterior1st: Exterior covering on house
* Exterior2nd: Exterior covering on house (if more than one material)
* MasVnrType: Masonry veneer type
* MasVnrArea: Masonry veneer area in square feet
* ExterQual: Exterior material quality
* ExterCond: Present condition of the material on the exterior
* Foundation: Type of foundation
* BsmtQual: Height of the basement
* BsmtCond: General condition of the basement
* BsmtExposure: Walkout or garden level basement walls
* BsmtFinType1: Quality of basement finished area
* BsmtFinSF1: Type 1 finished square feet
* BsmtFinType2: Quality of second finished area (if present)
* BsmtFinSF2: Type 2 finished square feet
* BsmtUnfSF: Unfinished square feet of basement area
* TotalBsmtSF: Total square feet of basement area
* Heating: Type of heating
* HeatingQC: Heating quality and condition
* CentralAir: Central air conditioning
* Electrical: Electrical system
* 1stFlrSF: First Floor square feet
* 2ndFlrSF: Second floor square feet
* LowQualFinSF: Low quality finished square feet (all floors)
* GrLivArea: Above grade (ground) living area square feet
* BsmtFullBath: Basement full bathrooms
* BsmtHalfBath: Basement half bathrooms
* FullBath: Full bathrooms above grade
* HalfBath: Half baths above grade
* Bedroom: Number of bedrooms above basement level
* Kitchen: Number of kitchens
* KitchenQual: Kitchen quality
* TotRmsAbvGrd: Total rooms above grade (does not include bathrooms)
* Functional: Home functionality rating
* Fireplaces: Number of fireplaces
* FireplaceQu: Fireplace quality
* GarageType: Garage location
* GarageYrBlt: Year garage was built
* GarageFinish: Interior finish of the garage
* GarageCars: Size of garage in car capacity
* GarageArea: Size of garage in square feet
* GarageQual: Garage quality
* GarageCond: Garage condition
* PavedDrive: Paved driveway
* WoodDeckSF: Wood deck area in square feet
* OpenPorchSF: Open porch area in square feet
* EnclosedPorch: Enclosed porch area in square feet
* 3SsnPorch: Three season porch area in square feet
* ScreenPorch: Screen porch area in square feet
* PoolArea: Pool area in square feet
* PoolQC: Pool quality
* Fence: Fence quality
* MiscFeature: Miscellaneous feature not covered in other categories
* MiscVal: $Value of miscellaneous feature
* MoSold: Month Sold
* YrSold: Year Sold
* SaleType: Type of sale
* SaleCondition: Condition of sale

In [ ]:
quantitative = [f for f in train.columns if train.dtypes[f] != 'object']
quantitative.remove('SalePrice')
quantitative.remove('Id')
qualitative = [f for f in train.columns if train.dtypes[f] == 'object']

<a id = '1.0'></a>
<p style = "font-size : 30px; color :#090f01 ;font-family : 'Chalkduster',fantasy;  "><strong>EDA :-</strong></p>

In [ ]:
train.dtypes

In [ ]:
from scipy import stats
from scipy.stats import norm, skew

sns.set_palette('pastel')
sns.distplot(train['SalePrice'] , fit=norm);

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(train['SalePrice'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

#Now plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution')

#Get also the QQ-plot
fig = plt.figure()
res = stats.probplot(train['SalePrice'], plot=plt)
plt.show() 

> The sale price value is right skewed. We need to make this normal distributed.

> the most corelated parameters in numerical values can be seen above plotting. And we can pick these as features for our macine learning model.

In [ ]:
missing = train.isnull().sum()
missing = missing[missing > 0]
missing.sort_values(inplace=True)
missing.plot.bar()

> 19 attributes have missing values, 5 over 50% of all data. Most of times NA means lack of subject described by attribute, like missing pool, fence, no garage and basement.

<a id = '1.0'></a>
<p style = "font-size : 30px; color :#090f01 ;font-family : 'Chalkduster',fantasy;  "><strong>Quantitative Variables:-</strong></p>

In [ ]:
f = pd.melt(train, value_vars=quantitative)
g = sns.FacetGrid(f, col="variable",  col_wrap=2, sharex=False, sharey=False)
g = g.map(sns.distplot, "value")

<a id = '1.0'></a>
<p style = "font-size : 30px; color :#090f01 ;font-family : 'Chalkduster',fantasy;  "><strong>Quanlitative Variables:-</strong></p>

In [ ]:
for c in qualitative:
    train[c] = train[c].astype('category')
    if train[c].isnull().any():
        train[c] = train[c].cat.add_categories(['MISSING'])
        train[c] = train[c].fillna('MISSING')

def boxplot(x, y, **kwargs):
    sns.boxplot(x=x, y=y)
    x=plt.xticks(rotation=90)
f = pd.melt(train, id_vars=['SalePrice'], value_vars=qualitative)
g = sns.FacetGrid(f, col="variable",  col_wrap=2, sharex=False, sharey=False, size=5)
g = g.map(boxplot, "value", "SalePrice")

Some categories seem to more diverse with respect to SalePrice than others. Neighborhood has big impact on house prices. Most expensive seems to be Partial SaleCondition. Having pool on property seems to improve price substantially. There are also differences in variabilities between category values.



In [ ]:
sns.lmplot(x='1stFlrSF',y='SalePrice',data=train) # 1stFlrSF seems very corelated with SalePrice.

In [ ]:
plt.scatter(x= 'GrLivArea', y='SalePrice', data = train)

Now lets convert all features into numeric/categorical data

In [ ]:
total = train.isnull().sum().sort_values(ascending=False)
percent = (train.isnull().sum()/train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(25)

In [ ]:
#to select appropriate efatures for training
k = 10 #number of variables for heatmap
plt.figure(figsize=(16,8))
corrmat = train.corr()
# picking the top 15 correlated features
cols = corrmat.nlargest(k, 'SalePrice')['SalePrice'].index
cm = np.corrcoef(train[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

In [ ]:
numerical=train.select_dtypes(include=['int64','float64'])
numerical_test=test.select_dtypes(include=['int64','float64'])
numerical=numerical.drop(['SalePrice','Id'],axis=1)
numerical_test=numerical_test.drop(['Id'],axis=1)
numerical

In [ ]:
numerical_categorical=pd.DataFrame()
numerical_continuous=pd.DataFrame()
i=0
while(i<36):
    x=numerical[numerical.columns[i]].value_counts().to_frame()
    if x.shape[0]<=20:
        numerical_categorical=pd.concat([numerical_categorical,numerical[x.columns[0]]],axis=1)
        i=i+1
    numerical_continuous=pd.concat([numerical_continuous,numerical[x.columns[0]]],axis=1)
    i=i+1
numerical_categorical

In [ ]:
numerical_categorical_test=pd.DataFrame()
numerical_continuous_test=pd.DataFrame()
i=0
while(i<numerical_categorical.shape[1]):
    numerical_categorical_test=pd.concat([numerical_categorical_test,numerical_test[numerical_categorical.columns[i]]],axis=1)
    i=i+1
i=0
while(i<numerical_continuous.shape[1]):
    numerical_continuous_test=pd.concat([numerical_continuous_test,numerical_test[numerical_continuous.columns[i]]],axis=1)
    i=i+1

In [ ]:
categorical=train.select_dtypes(include='object')
categorical_test=test.select_dtypes(include='object')


In [ ]:
categorical_cols=pd.concat([numerical_categorical,categorical],axis=1)
categorical_cols_test=pd.concat([numerical_categorical_test,categorical_test],axis=1)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
encoding=OneHotEncoder()
encoding.fit(categorical_cols)
x=encoding.transform(categorical_cols).toarray()
encoding_test=OneHotEncoder()
encoding.fit(categorical_cols_test)
x_test=encoding.transform(categorical_cols_test).toarray()

In [ ]:
x=pd.DataFrame(x)
x_test=pd.DataFrame(x_test)

In [ ]:
k=pd.DataFrame(np.zeros([1459,14]),columns=[str(i) for i in range(340,354)])
x_test=pd.concat([x_test,k],axis=1)
x_test

In [ ]:
X=pd.concat([x,numerical_continuous],axis=1)
X_test=pd.concat([x_test,numerical_continuous_test],axis=1)
X

In [ ]:
y=train['SalePrice'].ravel()
impute=SimpleImputer(strategy='median').fit_transform(X)
X=pd.DataFrame(impute)
X

<a id = '1.0'></a>
<p style = "font-size : 30px; color :#090f01 ;font-family : 'Chalkduster',fantasy;  "><strong>Modelling :-</strong></p>

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(n_estimators = 100, random_state = 0)
rfr.fit(X, y)

In [ ]:
rfr.score(X,y)